In [1]:
from os import listdir
from os.path import isfile, join
import string
import numpy as np
import pandas as pd
import math
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn import preprocessing
 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\monac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\monac\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
path="C:\\Users\\monac\\Downloads\\20_newsgroups\\20_newsgroups"
folders = [f for f in listdir(path)]

In [3]:
folders

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
categories = [ 'rec.sport.hockey','talk.politics.misc','sci.space','comp.graphics', 'sci.med']


In [5]:
files = []
for folder_name in categories:
    folder_path = join(path, folder_name)
    files.append([f for f in listdir(folder_path)])

In [6]:
sum(len(files[i]) for i in range(5))

5000

In [7]:
pathname_list = []
for fo in range(len(categories)):
    for fi in files[fo]:
        pathname_list.append(join(path, join(categories[fo], fi)))

In [8]:
for i in range(len(pathname_list)):
    print(pathname_list[i])

C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52550
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52551
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52552
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52553
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52554
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52555
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52556
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52557
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52558
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52559
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52560
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52561
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52562
C:\Users\mon

In [9]:
Y = []

for folder_name in categories:
    folder_path = join(path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)

In [10]:
len(Y)

5000

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
doc_train, doc_test, Y_train, Y_test = train_test_split(pathname_list, Y, random_state=0, test_size=0.30)

In [13]:
l= Y_train
dict(zip(set(l),map(l.count, set(l))))

{'sci.space': 704,
 'sci.med': 676,
 'talk.politics.misc': 701,
 'rec.sport.hockey': 716,
 'comp.graphics': 703}

In [14]:
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
Y_train= label_encoder.fit_transform(Y_train)
Y_test= label_encoder.fit_transform(Y_test)
 
print(Y_train)

[3 4 4 ... 4 3 3]


In [15]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'him', 'between', 'after', 'wouldn', 'needn', "wasn't", "don't", 'them', 'both', 'which', 'same', 'then', 'into', 'they', 'to', 'hadn', 'further', 'won', 'd', 'or', 'where', 'too', 'who', 't', 'ourselves', 'what', 'against', 's', "doesn't", 'just', 'by', 'haven', 'that', "it's", 'am', 'being', 'theirs', 'its', 'very', 'this', 'having', 'itself', "won't", 'hasn', "you'll", 'because', "isn't", 'if', 'yourselves', 'in', 'those', 've', 'yourself', 'wasn', 'ain', 'as', 'isn', 'these', 'hers', 'now', 'below', "you'd", 'is', 'some', "needn't", "didn't", 'all', 'during', 'through', "couldn't", 'ours', "you've", "aren't", 'ma', 'aren', 'don', 'do', 'me', 'for', 'under', 'o', 'were', 'again', 'couldn', 'has', 'our', 'off', 'before', 'nor', 'i', 'be', 'most', 'did', 'should', "shan't", 'you', 'each', 'so', 'but', 'm', 'didn', 'her', 'been', "weren't", 'can', 'herself', 'whom', 'over', 'such', "that'll", 'she', 'mightn', 'doing', 'my', 'and', 'other', 'on', 'at', 'shouldn', 'when', 'mustn', 'was'

In [16]:
len(stop_words)

179

In [17]:
stop_words =  stop_words.union({'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at',
 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 
 'can', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during',
 'each', 'few', 'for', 'from', 'further', 
 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's",
 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's",
 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself',
 "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself',
 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours' 'ourselves', 'out', 'over', 'own',
 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 
 'than', 'that',"that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", 
 "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 
 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
 "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's",'will', 'with', "won't", 'would', "wouldn't", 
 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 
 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'hundred', 'thousand', '1st', '2nd', '3rd',
 '4th', '5th', '6th', '7th', '8th', '9th', '10th'})

In [18]:
len(stop_words)

234

In [19]:
def remove_punctuation(text):
    text = " ".join(text)

    text = re.sub(r'[-!$%^&*#()_+|@\]\[~=`{}\\:"\';<>?,.\/]','',text)
    text = re.sub(r'[0-9]+','',text)
    
    text = text.strip()
    return text.split(" ")

def tokenize(text):
    text = word_tokenize(text)
    return text

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    text = [i for i in text if not i in stop_words]
    return text

lemmatizer = WordNetLemmatizer()
def lemmatize(text): 
    text = [lemmatizer.lemmatize(token[0]) for token in text]
    return text
 

def remove_whitespace(text):
    temp = list()
    for i in range(len(text)):
        if len(text[i].split()):
            temp.append(text[i].split())
    return temp

def preprocessing(text):
    text = text.lower()
    text = tokenize(text)
    text = remove_stopwords(text)
    text = remove_punctuation(text)
    text = remove_whitespace(text)
    text = lemmatize(text)
    return text

In [20]:

def remove_metadata(lines):
    for i in range(len(lines)):
        if(lines[i] == '\n'):
            start = i+1
            break
    new_lines = lines[start:]
    return new_lines

In [21]:
import joblib
import os
from collections import OrderedDict
#from natsort import natsorted
from pathlib import Path
def tokenize_final(path):
    file = open(path, "r",encoding ="utf8", errors ="surrogateescape")
 
    text_lines = file.read()

    
    
    doc_words = []
    
    
    
    doc_words= (preprocessing(text_lines))
#     print(doc_words)

    return doc_words

In [22]:


def flatten(list):
    new_list = []
    for i in list:
        for j in i:
            new_list.append(j)
    return new_list

In [23]:
len(categories)

5

In [24]:
list_of_words = []

for document in doc_train:
        list_of_words.append(tokenize_final(document))
        

In [25]:
i=0
for document in doc_train:
    print(Y_train[i],list_of_words[i])
    i+=1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4 ['xref', 'cantaloupesrvcscmuedu', 'talkpoliticsmisc', 'altactivism', 'altrushlimbaugh', 'misclegal', 'newsgroups', 'talkpoliticsmisc', 'altactivism', 'altrushlimbaugh', 'misclegal', 'path', 'cantaloupesrvcscmuedu', 'crabapplesrvcscmuedu', 'fsececmuedu', 'europaenggtefsdcom', 'howlandrestonansnet', 'gatech', 'purdue', 'mentorccpurdueedu', 'nooseecnpurdueedu', 'dynamoecnpurdueedu', 'garrod', 'garrod', 'dynamoecnpurdueedu', 'david', 'garrod', 'subject', 'waco', 'burning', 'messageid', 'cuevta', 'nooseecnpurdueedu', 'sender', 'news', 'nooseecnpurdueedu', 'usenet', 'news', 'organization', 'purdue', 'university', 'engineering', 'computer', 'network', 'date', 'wed', 'apr', 'gmt', 'line', 'interesting', 'sometimes', 'listen', 'u', 'news', 'seen', 'eye', 'another', 'country', 'bbc', 'world', 'news', 'service', 'shortwave', 'originating', 'london', 'report', 'survivor', 'waco', 'massacre', 'state', 'tank', 'making', 'hole', 'wall', 'building', 'knocked', 'kerosene', 'lamp', 'fire', 'started', 

In [26]:
len(list_of_words)

3500

In [27]:
print(i)

3500


In [28]:
len(flatten(list_of_words))

728709

In [29]:

total_words = set(np.asarray(flatten(list_of_words)))

In [30]:
type(stop_words)

set

In [31]:
print(total_words)

{'ultrasomethingorother', 'unfilled', 'friedman', 'ysebaertprimeausheppard', 'karelian', 'silbiger', 'artery', 'cobarruvias', 'transformed', 'mcb', 'bitorless', 'phoreceptors', 'tippett', 'ckwde', 'paradoxical', 'booksbodyofglass', 'spamdomuabedu', 'coegalon', 'impressed', 'netcombprofanecwhzhio', 'trottier', 'economical', 'kidding', 'typographical', 'amendment', 'violates', 'recartssfmarketplace', 'modificvations', 'samuesson', 'ditto', 'tonelli', 'swore', 'referral', 'laudace', 'iaga', 'eastwood', 'twominute', 'start', 'merge', 'dieing', 'biggest', 'uranian', 'filipe', 'custodian', 'rchui', 'dniklas', 'devnullphyspsuedu', 'cihmgs', 'catastrophically', 'forgive', 'geminigsfcnasagov', 'spacegroups', 'ptolemyastrowashingtonedu', 'kf', 'crumpling', 'outlet', 'originates', 'polydactyl', 'aggressiveness', 'mehl', 'mvbsaiccom', 'sexy', 'ammendment', 'internetgateway', 'eball', 'handgun', 'trian', 'surrounded', 'ccohen', 'coujia', 'rtime', 'admin', 'charonerusgsgov', 'unixhubcxjcia', 'oldest

In [32]:
len(total_words)

47551

In [33]:
def create_dataframe(list_of_words,total_words):
    df = []
    for i in list_of_words:
        wordDict = dict.fromkeys(total_words,0)
        for word in i:
            wordDict[word]+=1
        df.append(wordDict)
    
    return df
df = create_dataframe(list_of_words,total_words)
    

In [34]:
print(df[0])

{'ultrasomethingorother': 0, 'unfilled': 0, 'friedman': 0, 'ysebaertprimeausheppard': 0, 'karelian': 0, 'silbiger': 0, 'artery': 0, 'cobarruvias': 0, 'transformed': 0, 'mcb': 0, 'bitorless': 0, 'phoreceptors': 0, 'tippett': 0, 'ckwde': 0, 'paradoxical': 0, 'booksbodyofglass': 0, 'spamdomuabedu': 0, 'coegalon': 0, 'impressed': 0, 'netcombprofanecwhzhio': 0, 'trottier': 0, 'economical': 0, 'kidding': 0, 'typographical': 0, 'amendment': 0, 'violates': 0, 'recartssfmarketplace': 0, 'modificvations': 0, 'samuesson': 0, 'ditto': 0, 'tonelli': 0, 'swore': 0, 'referral': 0, 'laudace': 0, 'iaga': 0, 'eastwood': 0, 'twominute': 0, 'start': 0, 'merge': 0, 'dieing': 0, 'biggest': 0, 'uranian': 0, 'filipe': 0, 'custodian': 0, 'rchui': 0, 'dniklas': 0, 'devnullphyspsuedu': 0, 'cihmgs': 0, 'catastrophically': 0, 'forgive': 0, 'geminigsfcnasagov': 0, 'spacegroups': 0, 'ptolemyastrowashingtonedu': 0, 'kf': 0, 'crumpling': 0, 'outlet': 0, 'originates': 0, 'polydactyl': 0, 'aggressiveness': 0, 'mehl': 0,

In [35]:
df = pd.DataFrame(df)

In [36]:
df.loc[0]

ultrasomethingorother      0
unfilled                   0
friedman                   0
ysebaertprimeausheppard    0
karelian                   0
                          ..
affect                     0
mfoster                    0
sheba                      0
cagdj                      0
trig                       0
Name: 0, Length: 47551, dtype: int64

# Term Frequency

In [37]:
def term_frequency(df,Y,total_words):
    tf = pd.DataFrame(columns=total_words)
    categories =[0,1,2,3,4]
    for i in categories:
        
        df_category =pd.DataFrame(columns=total_words)
        for k in range(len(Y)):
            if Y[k] == i:
                df_category.loc[len(df_category.index)] = df.loc[k] 
            
        
        tf.loc[i] = df_category.sum(axis = 0, skipna = True)
        print(tf.loc[i])
    return tf    
                
tf = term_frequency(df,Y_train,total_words)
        
    
            
    
    

ultrasomethingorother      1
unfilled                   1
friedman                   0
ysebaertprimeausheppard    0
karelian                   0
                          ..
affect                     3
mfoster                    0
sheba                      0
cagdj                      0
trig                       1
Name: 0, Length: 47551, dtype: object
ultrasomethingorother       0
unfilled                    0
friedman                   14
ysebaertprimeausheppard     1
karelian                    0
                           ..
affect                      2
mfoster                     4
sheba                       2
cagdj                       0
trig                        0
Name: 1, Length: 47551, dtype: object
ultrasomethingorother       0
unfilled                    0
friedman                    2
ysebaertprimeausheppard     0
karelian                    1
                           ..
affect                     21
mfoster                     0
sheba                       0
cagdj

In [38]:
def compute_CF(tf,total_words):
    
    cfDict = {}
    N = len(total_words)
    
    
    cfDict = dict.fromkeys(total_words,0)
    for word in total_words:
        count =0
        for i in range(len(categories)):
            if(tf.iloc[i][word]>0):
                count+=1
        cfDict[word] = count
    return cfDict

cfDict = compute_CF(tf,total_words)
    
    
    
    
    
   

In [39]:
cfDict

{'ultrasomethingorother': 1,
 'unfilled': 1,
 'friedman': 4,
 'ysebaertprimeausheppard': 1,
 'karelian': 1,
 'silbiger': 1,
 'artery': 1,
 'cobarruvias': 1,
 'transformed': 4,
 'mcb': 1,
 'bitorless': 1,
 'phoreceptors': 1,
 'tippett': 1,
 'ckwde': 1,
 'paradoxical': 1,
 'booksbodyofglass': 1,
 'spamdomuabedu': 1,
 'coegalon': 1,
 'impressed': 5,
 'netcombprofanecwhzhio': 1,
 'trottier': 1,
 'economical': 1,
 'kidding': 4,
 'typographical': 1,
 'amendment': 4,
 'violates': 1,
 'recartssfmarketplace': 1,
 'modificvations': 1,
 'samuesson': 1,
 'ditto': 4,
 'tonelli': 1,
 'swore': 2,
 'referral': 1,
 'laudace': 1,
 'iaga': 1,
 'eastwood': 2,
 'twominute': 1,
 'start': 5,
 'merge': 3,
 'dieing': 1,
 'biggest': 5,
 'uranian': 1,
 'filipe': 2,
 'custodian': 1,
 'rchui': 1,
 'dniklas': 1,
 'devnullphyspsuedu': 1,
 'cihmgs': 1,
 'catastrophically': 1,
 'forgive': 5,
 'geminigsfcnasagov': 1,
 'spacegroups': 1,
 'ptolemyastrowashingtonedu': 1,
 'kf': 1,
 'crumpling': 1,
 'outlet': 3,
 'originat

In [40]:
def compute_ICF(cf,total_words):
    
    icfDict = {}
    N = len(total_words)
    
    
    icfDict = dict.fromkeys(total_words,0)
    for word in total_words:
        icfDict[word] = math.log(5/cf[word],2)
    return icfDict

icfDict = compute_ICF(cfDict,total_words)
    

In [41]:
icfDict

{'ultrasomethingorother': 2.321928094887362,
 'unfilled': 2.321928094887362,
 'friedman': 0.32192809488736235,
 'ysebaertprimeausheppard': 2.321928094887362,
 'karelian': 2.321928094887362,
 'silbiger': 2.321928094887362,
 'artery': 2.321928094887362,
 'cobarruvias': 2.321928094887362,
 'transformed': 0.32192809488736235,
 'mcb': 2.321928094887362,
 'bitorless': 2.321928094887362,
 'phoreceptors': 2.321928094887362,
 'tippett': 2.321928094887362,
 'ckwde': 2.321928094887362,
 'paradoxical': 2.321928094887362,
 'booksbodyofglass': 2.321928094887362,
 'spamdomuabedu': 2.321928094887362,
 'coegalon': 2.321928094887362,
 'impressed': 0.0,
 'netcombprofanecwhzhio': 2.321928094887362,
 'trottier': 2.321928094887362,
 'economical': 2.321928094887362,
 'kidding': 0.32192809488736235,
 'typographical': 2.321928094887362,
 'amendment': 0.32192809488736235,
 'violates': 2.321928094887362,
 'recartssfmarketplace': 2.321928094887362,
 'modificvations': 2.321928094887362,
 'samuesson': 2.32192809488

# Tf-ICF

In [42]:
def computeTFICF(tf,icfDict,total_words):
    tficf = pd.DataFrame(columns=total_words)
    for i in range(len(categories)):
        tficf_dict ={}
        for word in total_words:
            tficf_dict[word]= tf.iloc[i][word]*icfDict[word]
        tficf = tficf.append(tficf_dict, ignore_index = True)
    return tficf
tficf = computeTFICF(tf,icfDict,total_words)

In [43]:
tficf

,ultrasomethingorother,unfilled,friedman,ysebaertprimeausheppard,karelian,silbiger,artery,cobarruvias,transformed,mcb,...,bolivia,kadell,masterpiece,nordic,idee,affect,mfoster,sheba,cagdj,trig
0,2.321928,2.321928,0.000000,0.000000,0.000000,2.321928,0.000000,0.000000,0.643856,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.321928
1,0.000000,0.000000,4.506993,2.321928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.643856,2.321928,0.000000,0.0,9.287712,4.643856,0.000000,0.000000
2,0.000000,0.000000,0.643856,0.000000,2.321928,0.000000,6.965784,0.000000,0.965784,4.643856,...,2.321928,0.000000,0.000000,0.000000,4.643856,0.0,0.000000,0.000000,0.000000,3.965784
3,0.000000,0.000000,1.287712,0.000000,0.000000,0.000000,0.000000,0.000000,0.321928,0.000000,...,0.000000,0.000000,1.321928,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.643856,0.000000,0.000000,0.000000,0.000000,4.643856,0.321928,0.000000,...,0.000000,2.321928,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.321928,0.000000


# top k features

In [44]:
k=10
features = []
for i in range(0,5):
    index_list= sorted(range(len(tficf.loc[i])), key = lambda sub: tficf.loc[i][sub])[-k:]
    print(len(index_list))
    
    for j in index_list:
        print(tficf.loc[i][j])
        features.append(tficf.columns[j])
features = set(features)

10
195.04195997053841
199.61114232799173
211.508495181978
246.1243780580604
257.7340185324972
277.8360290006597
366.8646389922032
517.0165203891039
736.2286285720401
927.9935226109284
10
322.74800518934336
348.2892142331043
357.5769266126538
369.18656708709057
429.556697554162
482.5037546338873
571.1943113422911
879.082183100096
1000.7510088964531
1808.781985917255
10
222.90509710918678
263.0636908825851
276.3620978123273
280.24875611612083
290.24101186092025
357.5769266126538
369.18656708709057
399.3716323206263
450.45405040814825
688.3258649512366
10
278.63137138648347
280.24875611612083
292.5629399558076
301.8506523353571
305.3653899189807
309.5255495498066
337.05871534706836
392.4058480359642
419.0512060792939
645.4960103786867
10
341.32342994844225
370.13986656846146
399.3716323206263
420.26898517461257
436.5224818388241
485.2829718314587
543.3311742036427
796.4213365463652
907.8738851009587
1746.2670133462057


In [45]:
print(features)

{'compgraphicsanimation', 'patient', 'polygon', 'goalie', 'recsporthockey', 'playoff', 'jfif', 'vitamin', 'det', 'nhl', 'pittuucp', 'candida', 'clayton', 'espn', 'spacecraft', 'socmen', 'hockey', 'geb', 'stephanopoulos', 'jpeg', 'bruin', 'nsmca', 'orbit', 'baalke', 'gif', 'puck', 'fbi', 'optilink', 'talkpoliticsmisc', 'dyer', 'kelvinjplnasagov', 'coach', 'infection', 'comet', 'zootorontoedu', 'recfoodcooking', 'cspittedu', 'scispace', 'pixel', 'misclegal', 'quicktime', 'image', 'talkreligionmisc', 'cramer', 'compgraphics', 'tiff', 'sciastro', 'scimed', 'optilinkcom', 'sky'}


In [46]:
features = sorted(features)

# create test and train data

In [47]:
test_list_of_words = []

for document in doc_test:
        test_list_of_words.append(tokenize_final(document))
        
test_total_words = set(np.asarray(flatten(test_list_of_words)))

def create_test_dataframe(list_of_words,total_words):
    print(list_of_words)
    df = []
    for i in list_of_words:
       
        wordDict = dict.fromkeys(total_words,0)
        for word in i:
            if word in wordDict:
                wordDict[word]+=1
        df.append(wordDict)
    
    return df

test_df = create_test_dataframe(test_list_of_words,features)
test_df = pd.DataFrame(test_df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [48]:
len(test_df)

1500

In [49]:
features=list(features)

In [50]:
X_train = df.filter(features, axis=1)

In [51]:
X_train.shape

(3500, 50)

In [52]:
test_df.shape

(1500, 50)

In [53]:
X_train = np.asarray(X_train)
test_df =np.asarray(test_df)
Y_train = np.asarray(Y_train)
Y_test = np.asarray(Y_test)

# Naive Bayes

In [54]:
class MultiNB:
    def __init__(self,alpha=1):
        self.alpha = alpha
    
    def _prior(self): 
       
        P = np.zeros((self.n_classes_))
        _, self.dist = np.unique(self.y,return_counts=True)
        for i in self.classes_:
            P[i] = self.dist[i] / self.n_samples
        return P
            
    def fit(self, X, y): 
       
        self.y = y
        self.n_samples, self.n_features = X.shape
        self.classes_ = np.unique(y)
        self.n_classes_ = self.classes_.shape[0]
        self.class_priors_ = self._prior()
        
        # distinct values in each features
        self.uniques = []
        for i in range(self.n_features):
            tmp = np.unique(X[:,i])
            self.uniques.append( tmp )
            
        self.N_yi = np.zeros((self.n_classes_, self.n_features)) # feature count
        self.N_y = np.zeros((self.n_classes_)) # total count 
        for i in range(len(self.classes_)): 
            indices = np.argwhere(self.y==i).flatten()
            columnwise_sum = []
            for j in range(self.n_features):
                columnwise_sum.append(np.sum(X[indices,j]))
            
            self.N_yi[i] = columnwise_sum # 2d
            self.N_y[i] = np.sum(columnwise_sum) # 1d
            
    def _theta(self, x_i, i, h):
     
        
        Nyi = self.N_yi[h,i]
        Ny  = self.N_y[h]
        
        numerator = Nyi + self.alpha
        denominator = Ny + (self.alpha * self.n_features)
        
        return  (numerator / denominator)**x_i
    
    def _likelyhood(self, x, h):
    
        tmp = []
        for i in range(x.shape[0]):
            tmp.append(self._theta(x[i], i,h))
        
        return np.prod(tmp)
    
    def predict(self, X):
        samples, features = X.shape
        self.predict_proba = np.zeros((samples,self.n_classes_))
        
        for i in range(X.shape[0]):
            joint_likelyhood = np.zeros((self.n_classes_))
            
            for h in range(self.n_classes_):
                joint_likelyhood[h]  = self.class_priors_[h] * self._likelyhood(X[i],h) # P(y) P(X|y) 
                
            denominator = np.sum(joint_likelyhood)
            
            for h in range(self.n_classes_):
                numerator = joint_likelyhood[h]
                self.predict_proba[i,h] = (numerator / denominator)
            
        indices = np.argmax(self.predict_proba,axis=1)
        return self.classes_[indices]

In [57]:

nb = MultiNB(alpha=1)
nb.fit(X_train,Y_train)
y_pred = nb.predict(test_df)
me_score =accuracy_score(Y_test,y_pred)

print('Accuracy ',me_score)


Accuracy  0.994


In [58]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       297
           1       1.00      1.00      1.00       284
           2       1.00      0.99      0.99       324
           3       0.99      0.99      0.99       296
           4       1.00      1.00      1.00       299

    accuracy                           0.99      1500
   macro avg       0.99      0.99      0.99      1500
weighted avg       0.99      0.99      0.99      1500



In [59]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(Y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

Confusion Matrix

[[294   0   1   2   0]
 [  0 284   0   0   0]
 [  2   0 321   1   0]
 [  2   0   0 294   0]
 [  0   1   0   0 298]]
